In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Создадим data set, состоящий из данных о сотрудниках компании: возраст, количество проектов, часов в проекте, больничных дней и окладов

In [8]:
# Создаём сэмпл
n_samples = 500

# возраст сотрудников принимаем от 18 до 60
employee_age = np.random.choice(43, n_samples) + 18
# количество проектов, которые сотрудник выполняет в месяц
monthly_projects = np.random.choice(5, n_samples) + 1
# количество часов, которые сотрудник в среднем тратит на каждый проект
project_hours = np.random.choice(30, n_samples) + 10
# количесто больничных в меясц
sickleave_days = np.random.choice(7, n_samples) + 1
# зарплата сотрудников
salary = monthly_projects * project_hours * 3000 + sickleave_days * 1500

data = pd.DataFrame({'employee_age': employee_age, 'monthly_projects': monthly_projects, 
                     'project_hours': project_hours, 'sickleave_days': sickleave_days,
                     'salary': salary})
data.head(5)

,employee_age,monthly_projects,project_hours,sickleave_days,salary
0,54,5,15,7,235500
1,50,5,32,4,486000
2,29,5,36,1,541500
3,33,4,15,3,184500
4,57,5,12,5,187500


Цель упражнения - определить, от каких показателей может зависеть зарплата сотрудников, используя модель линейной регрессии.
Сперва в модели рассматриваем все параметры.

In [10]:
from sklearn.metrics import mean_absolute_error

X = data[['employee_age', 'monthly_projects', 'project_hours', 'sickleave_days']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['employee_age', 'monthly_projects', 'project_hours', 'sickleave_days']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [   86.6441392  70437.59581311  8884.47963591  1404.30620398]
Bias: -211556.07179487753
Error: 27384.572459485727


Получилось, что ошибка равна 27384.5. Ошибка большая, поэтому попробуем улучшить модель.

Предположим, что больничные никак не влияют на зарплату сотрудника, ведь он просто не работает в эти дни.

In [11]:
X = data[['employee_age', 'monthly_projects', 'project_hours']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['employee_age', 'monthly_projects', 'project_hours']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [   82.09399197 70426.50334953  8858.26884802]
Bias: -205231.16569948196
Error: 27795.42066866779


Ошибка немного увеличилась, значит, больничные все же немного влияют на зарплату.

Предположим, что больше всего на зарплату сотрудников влияет общее время работы сотрудников за месяц (новый признак), для этого умножим среднее количество часов потраченных на проект, на количество проектов, сделанных за месяц.

In [13]:
# создаем новый признак
data['hours_sum'] = data['monthly_projects'] * data['project_hours']
data.head()

,employee_age,monthly_projects,project_hours,sickleave_days,salary,hours_intotal,hours_sum
0,54,5,15,7,235500,75,75
1,50,5,32,4,486000,160,160
2,29,5,36,1,541500,180,180
3,33,4,15,3,184500,60,60
4,57,5,12,5,187500,60,60


In [14]:
X = data[['hours_sum']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['hours_sum']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [2996.4899505]
Bias: 6110.3967654409935
Error: 2628.0064774566235


Ошибка на порядок уменьшилась, значит, предположение было правильным и зарплата в основном зависит от общего количества рабочих часов. Однако ошибка всё еще довольно большая - 2628. Учтем первое предположение, что зарплата зависит в том числе и от количества больничных и добавим данный параметр в модель линейной регрессии.

In [16]:
X = data[['hours_sum', 'sickleave_days']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['hours_sum', 'sickleave_days']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [3000. 1500.]
Bias: -1.1641532182693481e-10
Error: 6.117625162005424e-11


Предположение было верным, ошибка стремиться к 0, значит, модель точно предсказывает зарплату сотрудника исходя из имеющихся данных.